<a href="https://colab.research.google.com/github/haytham918/low-rank-expectile/blob/main/Low_Rank_Expectile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as plot


In [56]:
df= pd.read_csv("/content/drive/MyDrive/low-rank-expectile/heartrate_seconds_merged.csv")
# Convert the 'Time' column to datetime format
df['Time'] = pd.to_datetime(df['Time'])


In [72]:
user_tenmin_df = df.groupby(['Id', pd.Grouper(key='Time', freq='5T')])['Value'].mean().unstack()
user_tenmin_matrix = user_tenmin_df.values
# print(user_tenmin_df)

In [73]:
nan_matrix = np.isnan(user_tenmin_matrix)
print("Matrix Entry Number: ", user_tenmin_matrix.shape[0] * user_tenmin_matrix.shape[1])
print("Nan Count: ", np.sum(nan_matrix))

Matrix Entry Number:  123718
Nan Count:  56200


In [76]:
import torch
from sklearn.model_selection import train_test_split

In [77]:
# Split the data into trainig/validation and exclude missing values
train_data, val_data, train_mask, val_mask = train_test_split(user_tenmin_matrix, nan_matrix, test_size=0.2, random_state=445)

